# OpenFOAM Slice Analysis

This program extracts data from Paraview slices of grids, interpolates them, and computes L2 norm, etc.
    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 08/11/2022
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia


In [1]:
# Load modules
import numpy as np
import glob, gzip, tqdm, os

X0 slices are computed by taking a plane parallel to the nozzle axis at the symmetry line of the domain.

All other slices are computed by taking a plane normal to the nozzle axis at a fixed streamwise station.

Computation process is the same for all planes:
1. Take threshold yMean <= 0.99 (removing pure air regions)
2. Transform geometry to ensure (0,0,0) is the middle of the nozzle exit for all grids
3. Take a slice
4. Compute a new field 'noz' which reflects which grid we are using
5. Merge all datasets for noz1...noz9 and save this for a single fluid.

## Function to read slice CSV

In [2]:
def read_slice_csv(filename):
    if filename[-3:].lower() == '.gz': fileReader=gzip.open
    else: fileReader=open
           
    print("Scanning "+filename+"...")
    with fileReader(filename,'r') as F:
        Nlines=len(F.readlines())
    
    with fileReader(filename,'r') as F:
        print("Reading data...")
        varNames = F.readline().strip().replace('"','').split(',')
        data = np.ndarray((len(varNames),Nlines-1))
        for n in tqdm.tqdm(range(Nlines-1)):
            d = F.readline().strip().replace('"','').split(',')
            data[:,n] = [float(d_) for d_ in d]
        
    return data, varNames

In [3]:
#topLevel = "/mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/slices/"
topLevel = "/mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/convergence/"

D,V=read_slice_csv(topLevel+"conv_fromVtm.csv")
print("dataset size: "+str(D.shape))

Scanning /mnt/internal-hdd/2021_pmdi/newGeomTrial/postProcessing/convergence/conv_fromVtm.csv...
Reading data...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 510712/510712 [00:14<00:00, 34182.53it/s]

dataset size: (66, 510712)


## Sort data combined in single CSV using a flag field
if GroupDatasets was used in Paraview with some field being a flag to indicate which data points are from which grids, we can split them apart.

In [4]:
sortingKey = 'resolution'

# Get the possible values of the sorting key and which column it is in.
keyIndex = V.index(sortingKey)
keyValues = np.unique(D[keyIndex,:])

# Find the largest number of gridpoints for any possible sorting key value
npts = 0
nmin = np.inf
kmin = None
for k in keyValues:
    n = np.sum(D[keyIndex,:]==k)
    if n>npts:
        npts=n
    if n<nmin: 
        nmin=n
        kmin=k
    print("Sorting index [%i] : %i gridpoints" %(k,n) )
    
# Move data to new array.
Ds = np.ndarray((len(keyValues), len(V), npts))
Ds[...] = np.nan
for j in tqdm.tqdm(range(len(keyValues))):
    i = D[keyIndex,:]==keyValues[j]
    Ds[j, :, :np.sum(i)] = D[:, i]


print("Moved %s -> %s" % (D.shape, Ds.shape))

Sorting index [1] : 39067 gridpoints
Sorting index [2] : 57278 gridpoints
Sorting index [3] : 127523 gridpoints
Sorting index [4] : 133630 gridpoints
Sorting index [5] : 153214 gridpoints


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.49it/s]

Moved (66, 510712) -> (5, 66, 153214)


## Interpolate data onto a uniform grid

In [5]:
import scipy.interpolate, scipy.spatial


def interpolateFunction(d, xy_input_indices, xy_output, pbar=None):
    """ Dan's function to run the interpolation on the last axis of a 2d array with the 1st axis
    being independent variables."""
    
    # Get coordinates of input data
    valid = ~np.isnan(d[xy_input_indices[0],:]) # X is not NaN
    xy_input = [ d[i,valid] for i in xy_input_indices ]
    
    # Precompute Delaunay triangulation - will be same for every variable.
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Delaunay.html
    points = scipy.spatial.Delaunay(np.transpose(np.array(xy_input)), furthest_site=False,\
                                    incremental=False, qhull_options=None)
    
    # Loop variables
    interpolatedResult = []
    for i in range(d.shape[0]): 
        if not i in xy_input_indices: # Interpolate data for non coord variable
            #https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.LinearNDInterpolator.html            
            fInt = scipy.interpolate.LinearNDInterpolator(points, d[i,valid], fill_value=np.nan, rescale=False)
            interpolatedResult.append(fInt(xy_output[0], xy_output[1]))
            
        elif i == xy_input_indices[0]: # Copy coordinates directly
            interpolatedResult.append(xy_output[0])
        elif i == xy_input_indices[1]: # Copy coordinates directly
            interpolatedResult.append(xy_output[1])
        else:
            raise IndexError
            
        if pbar is not None: pbar.update(d.shape[-1])
        
    return np.array(interpolatedResult)


In [10]:
# Specify the variables containing the coordinates of grid points
xv = "Center:0" ; yv = "Center:1" ; zv = "Center:2"
xi = V.index(xv); yi = V.index(yv); zi = V.index(zv)

# Use the lowest resolution grid as the basis for interpolation.
j_lowest = np.where(keyValues==kmin)[0][0]
X = Ds[j_lowest,xi,:]
Y = Ds[j_lowest,yi,:]
Z = Ds[j_lowest,zi,:]

X = X[~np.isnan(X)]
Y = Y[~np.isnan(Y)]
Z = Z[~np.isnan(Z)]

# Loop through X,Y,Z and figure out which to put in x and y
# Presently we assume that the plane is in X, Y or Z normal axis
# If it is tilted in all 3 axes we'd need to have a new coordinate system calculated/stored!
if np.nanstd(X)==0:
    xyz_input_indices = (yi,zi)
    xyz_output = (Y,Z)
    plane_norm = xv
    print("Plane normal is X axis")
elif np.nanstd(Y)==0:
    xyz_input_indices = (xi,zi)
    xyz_output = (X,Z)
    plane_norm = yv
    print("Plane normal is Y axis")
elif np.nanstd(Z)==0:
    xyz_input_indices = (xi,yi)
    xyz_output = (X,Y)
    plane_norm = zv
    print("Plane normal is Z axis")
else:
    raise ValueError("Error: plane is tilted in all axes, recalculate "+xv)
    
# Make new array of smaller size
Di = np.ndarray((len(keyValues), len(V), nmin))
Di[...] = np.nan

# Do interpolation for each grid
print("Interpolating...")
pbar = tqdm.tqdm(total=np.product(Ds.shape))
for j in range(len(keyValues)):
    Di[j,...] = interpolateFunction(Ds[j,...], xyz_input_indices, xyz_output, pbar )
pbar.close()

Plane normal is X axis
Interpolating...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50560620/50560620 [00:56<00:00, 893297.62it/s]


## Save the interpolated data for posterity

In [11]:
import h5py
with h5py.File(topLevel+"conv_interpolated.h5",'w') as H:
    vds = H.create_dataset("Variables",data=V)
    vds.attrs['sortingKey']=sortingKey
    H.create_dataset("Sorted data",data=Ds,compression='gzip')
    G=H.create_group("Interpolated data")
    G.create_dataset("coords",data=xyz_output,compression='gzip')
    G.create_dataset("data",data=Di,compression='gzip')
    G.attrs['plane_norm']=plane_norm
print("saved.")

saved.
